In [1]:
import os
import re
import pandas as pd
import numpy as np

In [2]:
def change_dtype_ser(ser):
    
    if ser.dtype == int:
        return ser.astype(np.int32)
    
    if ser.dtype == float:
        return ser.astype(np.float32)
    
    if ser.dtype == np.object:
        return ser.astype("category")
    
    return ser
    

def change_dtype_df(df):
    """
    change types of columns to reduce memory size
    :param df: dataframe
    :return df: dataframe
    """
    df["fecha_dato"] = pd.to_datetime(df["fecha_dato"])
    df["fecha_alta"] = pd.to_datetime(df["fecha_alta"])
    
    memory = df.memory_usage().sum() / 10**6
    print("Memory usage before changing types %0.2f MB" % memory)

    for col in df.columns:
        df[col] = change_dtype_ser(df[col])

    memory = df.memory_usage().sum() / 10 ** 6
    print("Memory usage after changing types %0.2f MB" % memory)
    return df


def load_csv(filename):
    df = pd.read_csv(filename)
    df = change_dtype_df(df)
    return df

In [3]:
INP_DIR = "data/data_/"

In [4]:
df_train = load_csv(os.path.join(INP_DIR, "train_cleaned.csv"))
df_test = load_csv(os.path.join(INP_DIR, "test_cleaned.csv"))

/home/hai/opt/anaconda3-2020.02/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Memory usage before changing types 6946.48 MB
Memory usage after changing types 3302.66 MB
Memory usage before changing types 116.20 MB
Memory usage after changing types 49.28 MB


In [5]:
# drop this column becuase it is too imbalanced
df_train = df_train.drop(["ind_empleado"], axis=1)
df_test = df_test.drop(["ind_empleado"], axis=1)

In [6]:
# days from when the data is recorded 
df_train["fecha_alta"] = (df_train["fecha_alta"] - df_train["fecha_dato"]).dt.days
df_test["fecha_alta"] = (df_test["fecha_alta"] - df_test["fecha_dato"]).dt.days

In [7]:
df_train.isnull().sum().sum(), df_test.isnull().sum().sum()

(0, 0)

In [8]:
df_train.shape, df_test.shape

((13647309, 67), (929615, 19))

In [9]:
df_train.columns

Index(['fecha_dato', 'ncodpers', 'pais_residencia', 'sexo', 'age',
       'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes',
       'tiprel_1mes', 'indresi', 'indext', 'canal_entrada', 'indfall',
       'cod_prov', 'ind_actividad_cliente', 'renta', 'segmento',
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1',
       'ind_ahor_fin_ult1_NEW_PUR', 'ind_aval_fin_ult1_NEW_PUR',
       'ind_cco_fin_ult1_NEW_PUR', 'ind_cder_fin_ult1_NEW_PUR',
       'ind_cno_fin_ult1_NEW_PUR', 'ind_ctj

In [10]:
df_test.columns

Index(['fecha_dato', 'ncodpers', 'pais_residencia', 'sexo', 'age',
       'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes',
       'tiprel_1mes', 'indresi', 'indext', 'canal_entrada', 'indfall',
       'cod_prov', 'ind_actividad_cliente', 'renta', 'segmento'],
      dtype='object')

In [11]:
PROD_COLS = [col for col in df_train.columns if re.match(r"^ind_.*_ult1$", col)]
print(PROD_COLS)

['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']


In [12]:
NEW_PURCH_COLS = [col for col in df_train.columns if re.match(r"^ind_.*_ult1_NEW_PUR$", col)]
print(NEW_PURCH_COLS, "\n")

prod_popul = df_train[NEW_PURCH_COLS].sum(axis=0)
prod_popul = prod_popul.sort_values(ascending=False)/prod_popul.sum() * 100
print(prod_popul, "\n")

SIXTEEN_MOST_POP_PROD = prod_popul.sort_values(ascending=False).index[:16].to_list()
print(SIXTEEN_MOST_POP_PROD)

['ind_ahor_fin_ult1_NEW_PUR', 'ind_aval_fin_ult1_NEW_PUR', 'ind_cco_fin_ult1_NEW_PUR', 'ind_cder_fin_ult1_NEW_PUR', 'ind_cno_fin_ult1_NEW_PUR', 'ind_ctju_fin_ult1_NEW_PUR', 'ind_ctma_fin_ult1_NEW_PUR', 'ind_ctop_fin_ult1_NEW_PUR', 'ind_ctpp_fin_ult1_NEW_PUR', 'ind_deco_fin_ult1_NEW_PUR', 'ind_deme_fin_ult1_NEW_PUR', 'ind_dela_fin_ult1_NEW_PUR', 'ind_ecue_fin_ult1_NEW_PUR', 'ind_fond_fin_ult1_NEW_PUR', 'ind_hip_fin_ult1_NEW_PUR', 'ind_plan_fin_ult1_NEW_PUR', 'ind_pres_fin_ult1_NEW_PUR', 'ind_reca_fin_ult1_NEW_PUR', 'ind_tjcr_fin_ult1_NEW_PUR', 'ind_valo_fin_ult1_NEW_PUR', 'ind_viv_fin_ult1_NEW_PUR', 'ind_nomina_ult1_NEW_PUR', 'ind_nom_pens_ult1_NEW_PUR', 'ind_recibo_ult1_NEW_PUR'] 

ind_recibo_ult1_NEW_PUR      27.197280
ind_nom_pens_ult1_NEW_PUR    15.048197
ind_nomina_ult1_NEW_PUR      13.101134
ind_cco_fin_ult1_NEW_PUR     12.426018
ind_tjcr_fin_ult1_NEW_PUR    12.304237
ind_cno_fin_ult1_NEW_PUR      6.601516
ind_ecue_fin_ult1_NEW_PUR     4.682679
ind_dela_fin_ult1_NEW_PUR     2.2557

In [13]:
DEMOG_COLS = [col for col in df_train.columns 
              if col not in PROD_COLS + NEW_PURCH_COLS + ["fecha_dato", "ncodpers"]]
print(DEMOG_COLS)

['pais_residencia', 'sexo', 'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 'canal_entrada', 'indfall', 'cod_prov', 'ind_actividad_cliente', 'renta', 'segmento']


In [14]:
def _is_unique(x):
    return len(x) == x.nunique()


def extract_subset(df, row_filter, cols):
    df = df.copy()
    return df.loc[row_filter, cols]


def extract_y(df, timestamp, y_cols=SIXTEEN_MOST_POP_PROD):
    # only use row when customer buys exactly one product
    any_new_pur = df[y_cols].sum(axis=1) == 1
    row_filter = (df["fecha_dato"] == timestamp) & any_new_pur
    cols = ["ncodpers"] + y_cols
    
    df_out = extract_subset(df, row_filter, cols)
    assert _is_unique(df_out["ncodpers"]), "ncodpers must be unique"
    
    df_out.columns = [col.replace("_NEW_PUR", "") if col.endswith("_NEW_PUR") else col for col in df_out.columns]
    #df_out = df_out.set_index("ncodpers")
    return df_out


def extract_x_demog(df, timestamp, customer_ids, demog_cols=DEMOG_COLS):
    row_filter = (df["fecha_dato"] == timestamp) & df["ncodpers"].isin(customer_ids)
    cols = ["ncodpers"] + demog_cols
    
    df_out = extract_subset(df, row_filter, cols)
    assert _is_unique(df_out["ncodpers"]), "ncodpers must be unique"

    return df_out


# This function can give dataframe having less rows than len(customer_ids)
# This is becuse number of row at lag_timestamp is not the same as at timestamp
def extract_y_prod_lag(df, timestamp_lag, customer_ids, suffix="_LAG", prod_cols=PROD_COLS):
    row_filter = (df["fecha_dato"] == timestamp_lag) & df["ncodpers"].isin(customer_ids)
    cols = ["ncodpers"] + prod_cols
    
    df_out = extract_subset(df, row_filter, cols)
    assert _is_unique(df_out["ncodpers"]), "ncodpers must be unique"
    
    df_out.columns = ["ncodpers"] + [col + suffix for col in df_out.columns if col != "ncodpers"]
    
    return df_out



def extract_X_y_train(df, timestamp, timestamp_lags):
    y_train = extract_y(df, timestamp)
    print("y_train.shape", y_train.shape)
    
    x_demog = extract_x_demog(df, timestamp, customer_ids=y_train["ncodpers"])
    print("x_demog.shape:", x_demog.shape)
    assert y_train.shape[0] == x_demog.shape[0], "x_demog must have the same number of rows as y_train"
    
    x_lags = []
    for t, lag in enumerate(timestamp_lags):
        assert pd.to_datetime(lag) < pd.to_datetime(timestamp), lag + " lag is not before timestamp " +  timestamp
        lag_label = "_LAG%d" % (t + 1)
        
        x_lag = extract_y_prod_lag(df, lag, customer_ids=y_train["ncodpers"], suffix=lag_label)
        print(lag, lag_label, x_lag.shape)
        x_lags.append(x_lag)
    
    X_train = y_train[["ncodpers"]].merge(x_demog, how="left", on="ncodpers")
    print("Nulls after merging y and x_demog:", X_train.isnull().sum().sum())
    
    for t, x_lag in enumerate(x_lags):
        X_train = X_train.merge(x_lag, how="left", on="ncodpers")
        print("Nulls at %d:" %(t + 1), X_train.isnull().sum().sum())
    
    print("X_train.shape:", X_train.shape)
    print("y_train.shape:", y_train.shape)
    return X_train, y_train


def extract_X_test(train, test, timestamp, timestamp_lags):
    x_demog = extract_x_demog(test, timestamp, customer_ids=test["ncodpers"])
    print("x_demog.shape:", x_demog.shape)
    print("Nulls of x_demog:", x_demog.isnull().sum().sum())
    
    x_lags = []
    for t, lag in enumerate(timestamp_lags):
        assert pd.to_datetime(lag) < pd.to_datetime(timestamp), lag + " lag is not before timestamp " +  timestamp
        lag_label = "_LAG%d" % (t + 1)
        
        x_lag = extract_y_prod_lag(train, lag, customer_ids=test["ncodpers"], suffix=lag_label)
        print(lag, lag_label, x_lag.shape)
        x_lags.append(x_lag)
        
    X_test = x_demog
    for t, x_lag in enumerate(x_lags):
        X_test = X_test.merge(x_lag, how="left", on="ncodpers")
        print("Nulls at %d:" %(t + 1), X_test.isnull().sum().sum())
    
    print("X_test.shape:", X_test.shape)
    return X_test